In [41]:
source("../BrusselSprouts/scripts/functions.R")

In [42]:
samples=readLines("~/data/TRD/1011Matrix.gvcf.gz.samples")
length(samples)

[1] 1011

In [43]:
chris_crosses=data.frame(sample1=c("ACP","BAP","CCD","ATE","ACK","AKE","BAH","ANG"),
                        sample2=c("BFP","CMP","CPG","YCR","ACV","BAH","CGD","CEI"))
# for reference

In [44]:
manual_comparisons=data.frame(sample1=c("BAK","BAN"), sample2=c("BET","BTI"))

In [45]:
library(tidyr)

In [46]:
comparisons_random=expand.grid(sample1=samples,sample2=samples)
comparisons_random$dup=which(paste(comparisons_random$sample2,comparisons_random$sample1) == paste(comparisons_random$sample1,comparisons_random$sample2))
comparisons_random$nrow=1:nrow(comparisons_random)
comparisons_random=comparisons_random[comparisons_random$dup < comparisons_random$nrow,]
comparisons_random=comparisons_random[sample(1:nrow(comparisons_random), size=1000, replace = FALSE),]
nrow(comparisons_random)

[1] 1000

In [47]:
comparisons=bind_rows(chris_crosses,manual_comparisons,comparisons_random)
comparisons=unique(comparisons)
nrow(comparisons)

[1] 1010

In [48]:
#comparisons=expand.grid(sample1=samples,sample2=samples)
#comparisons$dup=which(paste(comparisons$sample2,comparisons$sample1) == paste(comparisons$sample1,comparisons$sample2))
#comparisons$nrow=1:nrow(comparisons)
#comparisons=comparisons[comparisons$dup < comparisons$nrow,]
#nrow(comparisons)

In [49]:
# these are too many comparisons. I should only do that for maybe 1000 or so, so the last filter step

Maybe we can just quickly subset using bcftools and count lines

````bash

bcftools view -s ACK,CPG -Ou 1011Matrix.SNPs.var.GQ20.DP5.Miss10.vcf.gz | bcftools view -g ^het -H --min-ac 1 --max-ac 3 | wc -l

`````


In [50]:
initial_timedate=Sys.time()
scripts_dir="/home/jnrunge/data/TRD/OH/scripts/"
jobname="Scev-OppHomo"
sbatch_list=paste(scripts_dir,jobname,"-sbatch.list", sep="")
print(sbatch_list)
if(file.exists(sbatch_list)){
    system(command=paste("rm -f ", sbatch_list, sep=""))
}

for(i in 1:nrow(comparisons)){
    sample1=comparisons$sample1[i]
    sample2=comparisons$sample2[i]
    if(!file.exists(paste("~/data/TRD/OM/",sample1,"-",sample2,".OH",sep="")))
    {
        sbatch=paste("#!/bin/bash
#SBATCH -p fast          # The account name for the job
#SBATCH --job-name=",jobname,"  # The job name
#SBATCH -o ",scripts_dir,"logs/",jobname,"-",sample1,",",sample2,".out
#SBATCH -e ",scripts_dir,"logs/",jobname,"-",sample1,",",sample2,".err
#SBATCH -c 1                 # The number of cpu cores to use
#SBATCH --time=11:59:00       # The time the job will take to run 
#SBATCH --mem=4gb

date
source ~/anaconda3/etc/profile.d/conda.sh
conda activate bwaetc

cd ~/data/TRD/

bcftools view -s ",sample1,",",sample2," -Ou 1011Matrix.SNPs.var.GQ20.DP5.Miss10.vcf.gz | bcftools view -g ^het -H --min-ac 1 --max-ac 3 | wc -l > OH/",sample1,"-",sample2,".OH

conda activate JupyteR4
Rscript ~/BrusselSprouts/scripts/Execute_Sbatches.R '",initial_timedate,"' ",sbatch_list," 20 ",jobname,"

date
",sep="")
        sbatch_file=paste("~/data/TRD/OH/scripts/",jobname,"-",sample1,"-",sample2,".sbatch",sep="")
        #print(sbatch_file)
        writeLines(sbatch, sbatch_file)

        write(sbatch_file,file=sbatch_list,append=TRUE)


            
        }
}


print(system(command=paste("wc -l ", sbatch_list, sep=""),intern=TRUE))



[1] "/home/jnrunge/data/TRD/OH/scripts/Scev-OppHomo-sbatch.list"
[1] "1010 /home/jnrunge/data/TRD/OH/scripts/Scev-OppHomo-sbatch.list"


In [51]:
print(system(command=paste("Rscript ~/BrusselSprouts/scripts/Execute_Sbatches.R '",initial_timedate,"' ",sbatch_list," 20 ",jobname, "30", sep=""), intern=TRUE))

 [1] "[1] \"Currently running 0 jobs.\""                  
 [2] "[1] \"1010 sbatches remaining! Running 21 more...\""
 [3] "[1] \"Submitted batch job 276876\""                 
 [4] "[1] \"Submitted batch job 276877\""                 
 [5] "[1] \"Submitted batch job 276878\""                 
 [6] "[1] \"Submitted batch job 276879\""                 
 [7] "[1] \"Submitted batch job 276880\""                 
 [8] "[1] \"Submitted batch job 276882\""                 
 [9] "[1] \"Submitted batch job 276884\""                 
[10] "[1] \"Submitted batch job 276886\""                 
[11] "[1] \"Submitted batch job 276888\""                 
[12] "[1] \"Submitted batch job 276890\""                 
[13] "[1] \"Submitted batch job 276892\""                 
[14] "[1] \"Submitted batch job 276894\""                 
[15] "[1] \"Submitted batch job 276896\""                 
[16] "[1] \"Submitted batch job 276898\""                 
[17] "[1] \"Submitted batch job 276900\""               

In [39]:
homofiles=system(command="ls ~/data/TRD/OH/*.OH",intern=TRUE)
length(homofiles)

[1] 610

In [40]:
homo_df=data.frame(file=homofiles, value=NA, stringsAsFactors = FALSE)
for(i in 1:nrow(homo_df)){
    homo_df$value[i]=as.numeric(readLines(homo_df$file[i]))
}

ERROR: Error in homo_df$value[i] <- as.numeric(readLines(homo_df$file[i])): replacement has length zero


In [ ]:
hist(homo_df$value)